In [1]:
import os 
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero

dir_ = '../../../data/FM_data/'
group_dir_ = '../../../data/FM_data/groups/low'

In [2]:
file_name = 'normalized_to_rating_filter_track_5_user_50.csv'
df = pd.read_pickle(os.path.join(dir_, file_name[:-3] + 'pkl'))
svd = pd.read_pickle(os.path.join(dir_, 'prediction_svd_top_N_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])

num_user = len(svd['uid'].unique())
num_user

218

In [3]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)

In [4]:
svd['count'] = 1
svd[:5]

,uid,tid,rating,count
0,0,2,2.154104,1
1,0,14,2.175915,1
2,0,15,2.244578,1
3,0,20,2.275416,1
4,0,21,2.088449,1


In [5]:
# Count POP to DF
tid_list = []
pop_list = []
for i in df['tid'].unique():
    tid_list.append(i)
    pop_list.append(len(df[df['tid']==i])/num_user*5)

In [6]:
d = {'tid': tid_list, 'rating': pop_list}
df_pop = pd.DataFrame(data=d)
df_pop = df_pop.sort_values(by=['rating'], ascending=False)
df_pop[:10]

,tid,rating
9147,9147,2.064220
55,55,2.064220
6190,6190,2.018349
555,555,1.972477
1388,1388,1.972477
3825,3825,1.949541
1360,1360,1.949541
48,48,1.949541
355,355,1.949541
5935,5935,1.880734


In [7]:
lambdas = []
for i in range(11):
    lambdas.append(i/10)
lambdas

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

In [18]:
p_list = []
r_list = []
for lambda_ in tqdm(lambdas):
    top_n_size = 4
    top_n_lists_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = df_pop.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
        top_n_lists_g.append(top_n_lists)   
        
    threshold = 0.0
    avg_precisions = []
    avg_recalls = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        precisions = []
        recalls = []
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]
            
            for k in range(len(group)):
                high_rating = 0
                for l in range(top_n_size):
                    uid = group[k]
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    if len(t) > 0 and t.iloc[0]['rating'] > threshold:
                        high_rating += 1

                precision = high_rating / top_n_size
                recall = high_rating / len(test[test['uid'] == uid])
                precisions.append(precision)
                recalls.append(recall)
            

        avg_precision = 0
        for precision in precisions:
            avg_precision += precision
        avg_precision /= len(precisions)
        avg_precisions.append(avg_precision)

        avg_recall = 0
        for recall in recalls:
            avg_recall += recall
        avg_recall /= len(recalls)
        avg_recalls.append(avg_recall)
        
#     print(avg_precisions)
#     print(avg_recalls)
    p_list.append(avg_precisions)
    r_list.append(avg_recalls)

[0.0775, 0.07037037037037037, 0.04807692307692308, 0.043333333333333335, 0.05303030303030303, 0.03571428571428571, 0.0375]
[0.0012398081150891513, 0.001482150307366794, 0.001334700784263855, 0.0013378777093685974, 0.0019521696548247553, 0.0014529617215746113, 0.0016751347675729236]
[0.135, 0.08333333333333333, 0.052884615384615384, 0.03666666666666667, 0.05303030303030303, 0.03571428571428571, 0.04375]
[0.0018421495979845008, 0.0014353057051327085, 0.000966785922021324, 0.0008914753138751068, 0.0014716121262144438, 0.0011982971923319458, 0.0018751347675729235]
[0.1975, 0.11296296296296296, 0.0673076923076923, 0.04666666666666667, 0.05303030303030303, 0.04081632653061224, 0.04375]
[0.0023824075363758036, 0.0018112910164483202, 0.0014742070772683995, 0.0010805100629098557, 0.0014716121262144438, 0.0012630850122218062, 0.0018751347675729235]
[0.20375, 0.14629629629629629, 0.08413461538461539, 0.06333333333333334, 0.06439393939393939, 0.04591836734693878, 0.05]
[0.0023010542262762736, 0.00

In [19]:
for i in p_list:
    print(i)

for i in r_list:
    print(i)

[0.0775, 0.07037037037037037, 0.04807692307692308, 0.043333333333333335, 0.05303030303030303, 0.03571428571428571, 0.0375]
[0.135, 0.08333333333333333, 0.052884615384615384, 0.03666666666666667, 0.05303030303030303, 0.03571428571428571, 0.04375]
[0.1975, 0.11296296296296296, 0.0673076923076923, 0.04666666666666667, 0.05303030303030303, 0.04081632653061224, 0.04375]
[0.20375, 0.14629629629629629, 0.08413461538461539, 0.06333333333333334, 0.06439393939393939, 0.04591836734693878, 0.05]
[0.2075, 0.14814814814814814, 0.09375, 0.08333333333333333, 0.06818181818181818, 0.05612244897959184, 0.05]
[0.21125, 0.1425925925925926, 0.09615384615384616, 0.08333333333333333, 0.08333333333333333, 0.0663265306122449, 0.0625]
[0.2, 0.14629629629629629, 0.0889423076923077, 0.08, 0.07954545454545454, 0.0663265306122449, 0.05625]
[0.20375, 0.14074074074074075, 0.09615384615384616, 0.08333333333333333, 0.09090909090909091, 0.07653061224489796, 0.05625]
[0.195, 0.13518518518518519, 0.10336538461538461, 0.083

In [20]:
def dcg_at_k(r, k, method=0):
    r = np.asfarray(r)[:k]
    for i in range(len(r)):
        r[i] = 2**r[i] -1
    
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.


def ndcg_at_k(r, r_max, k, method=0):
    dcg_max = dcg_at_k(r_max, k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

In [21]:
for lambda_ in tqdm(lambdas):
    top_n_lists_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = df_pop.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
        top_n_lists_g.append(top_n_lists)   
        
#     print('lambda = ' + str(lambda_))
    threshold = 0.0
    avg_nDCGs = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        nDCGs = []
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]
            
            for k in range(len(group)):
                truth_rating = []
                uid = group[k]
                for l in range(top_n_size):
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    if len(t) > 0:
                        truth_rating.append(t.iloc[0]['rating'])
                    else:
                        truth_rating.append(0)
                max_rating = test[test['uid']==uid].sort_values(by=['rating'],  ascending=False)['rating'].values[:top_n_size]
                nDCG = ndcg_at_k(truth_rating, max_rating, top_n_size, method=1)
                nDCGs.append(nDCG)  
                
        avg_nDCG = 0
        for nDCG in nDCGs:
            avg_nDCG += nDCG
        avg_nDCG /= len(nDCGs)
        avg_nDCGs.append(avg_nDCG)
        
    print(avg_nDCGs)
#     print('----------------------------------------') 

[0.02882698262296474, 0.028543470919542194, 0.019119374832316245, 0.01613062416754107, 0.01927737464639077, 0.01056404215878583, 0.0010342947194397834]
[0.059591853230239335, 0.035907393212003674, 0.021288772386776993, 0.01425337064970261, 0.02417191087635331, 0.009125568824680217, 0.0011698301985993405]
[0.09084069012674158, 0.05319135287996382, 0.0252091766872615, 0.014603971452413766, 0.024775093417532083, 0.012879435326041737, 0.0013358398582499896]
[0.0938234594485554, 0.06557308550542525, 0.03798168474619204, 0.02195604120073409, 0.030093580076929274, 0.011822595537680933, 0.00206570109000598]
[0.09501850411475815, 0.06365053210564177, 0.040444578822260116, 0.028971170717893013, 0.036468828851192625, 0.016829311962649637, 0.0019959669638178103]
[0.09769985509381474, 0.060918891539409235, 0.04110466734778538, 0.029222949081264257, 0.040954706010559914, 0.02108780026220001, 0.009820864602983651]
[0.09561210858560408, 0.06232454144991722, 0.03904848179469268, 0.027801494429194436, 0